In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import csv

In [2]:
#spw = samples per window
def normalize(file, out, spw, label_col_start, scaler):
    #get data from file
    print("Reading data from: ", file)
    data = pd.read_csv(file,sep=',',header=0,dtype=np.float64)

    # get features columns 
    x_data = data.iloc[:, 0:label_col_start] # Tutto tranne le ultime 4 colonne
    # get label column
    y_data = data.iloc[:,label_col_start:] # Le ultime 4 colonne contengono i basografici
    # print(y_data)
    print("x_data shape:")
    print(x_data.shape)
    
    # ge features number
    n_features=int(x_data.shape[1]/spw)
    #init data
    data = np.zeros((x_data.shape[0],x_data.shape[1]))

    for i in range(n_features):
        feat = scaler.fit_transform(x_data.iloc[:,i::n_features])
        for l in range(feat.shape[1]):
            data[:,(l*n_features)+i] = feat[:,l]
          #  data[:,(l*n_features)+i] = feat[:,l]
    data = np.hstack((data,y_data))
    print("Writing normalized data to: ", out)
    np.savetxt(out, data, fmt='%.6f', delimiter=",")

def normalize_subjects(path_in, prefix, path_out, suffix, n_subjects, label_col_start, spw, scaler):
    #Normalization 
    if not os.path.exists(path_out):
        os.makedirs(path_out)
    for j in range(1,n_subjects+1):
        normalize((path_in + prefix + str(j) + suffix + '.csv'),path_out + prefix + str(j) + suffix + '_norm.csv',spw, label_col_start, scaler)

In [5]:
scaler = MinMaxScaler()
#scaler = StandardScaler()
#normalize_subjects("subjects/bpf-20-450_rect/", "s_bpf-20-450_rect_", "subjects/bpf-20-450_rect/min-max/", "_norm",26, -1, scaler)
normalize_subjects("subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/", "s_", "subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/", "_windows_200_features",26, -1, 1, scaler)
#normalize_subjects("../subjects/", "s", "../subjects/min-max/", "_norm",105, -4)

Reading data from:  subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/s_1_windows_200_features.csv
x_data shape:
(24002, 40)
Writing normalized data to:  subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/s_1_windows_200_features_norm.csv
Reading data from:  subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/s_2_windows_200_features.csv
x_data shape:
(24265, 40)
Writing normalized data to:  subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/s_2_windows_200_features_norm.csv
Reading data from:  subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/s_3_windows_200_features.csv
x_data shape:
(23996, 40)
Writing normalized data to:  subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/s_3_windows_200_features_norm.csv
Reading data from:  subjects/bpf-20-450/windows_200-del_tr-False-slide-True-digits-3-pace-20/s_4_windows_200_features.csv
x_data shape:
(24003, 40)


In [5]:
# Questa serve per controllare se la normalizzazione è andata bene
# Controlla per ogni file, per ogni colonna, che il minimo sia 0 e il massimo sia 1
# Se sì, vuol dire che la normalizzazione è andata a buon fine
path = "../subjects/min-max/"
suffix = "_norm"

def minmax(row, i, min, max):
    if (float(row[i]) > max[i]): max[i] = float(row[i])
    if (float(row[i]) < min[i]): min[i] = float(row[i])

        
def getMinMax(path, suffix, limit):
    for subj in range(1,limit+1):
        with open(path + 's' + str(subj) + suffix + '.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            print("Subject:" + str(subj))
            min = [1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000 ,1000000,1000000,1000000,1000000]
            max = [-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000,-1000000]
            for row in csv_reader:
                for i in range(0,12):
                    minmax(row,i, min, max)
            print("Min: " + str(min))
            print("Max: " + str(max))        

getMinMax(path,suffix,26)    # L'ultimo argomento rappresenta il numero di soggetti su cui eseguire il test

Subject:1
Min: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Subject:2
Min: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Subject:3
Min: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Subject:4
Min: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Subject:5
Min: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Subject:6
Min: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Subject:7
Min: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Max: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Subjec